<a href="https://colab.research.google.com/github/as3fn/Coursera_Capstone/blob/master/IBM_Capstone_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1**

In this part the address of Ontareo is scraped from the internet and incoded into a dataframe. 

In [1]:
from bs4 import BeautifulSoup
import requests
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
file=requests.get(wiki)
sop=BeautifulSoup(file.content,'html.parser')
table=sop.find('table',class_="wikitable sortable")


type(table)

bs4.element.Tag

In [2]:
import re
tb=[]
for row in table.find_all('tr'):
    for cell in row.find_all('td'):
        #tb.append(re.match(r'\w+[\']?\s?\w+',cell.text).group())
        tb.append(cell.text.replace('\n',''))


tb[:6]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned']

In [0]:
from collections import defaultdict
final_table=defaultdict(list)

for i,h in enumerate(table.find_all('th')):
    final_table[h.text.replace('\n','')]=tb[i::3][:-1] #the parsere produced \n at the end of the strings at the last location in the row

In [4]:
import pandas as pd

final_table=pd.DataFrame(final_table)
final_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
final_table[(final_table['Borough']!='Not assigned')& (final_table['Neighbourhood']=='Not assigned')]



,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


In [6]:
mask=(final_table['Borough']=='Not assigned')
final_table.drop(final_table[mask].index,inplace=True)

final_table[final_table['Neighbourhood']=='Not assigned']['Neighbourhood'].index
final_table.loc[9,'Neighbourhood']=final_table.loc[9,'Borough']


final_table=final_table.groupby(['Postcode','Borough']).agg(lambda x:','.join(x))
final_table.reset_index(inplace=True)

final_table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Part 2.**

In this part the location information about each postcode is incoded and the two dataframes are meged together

In [7]:
!wget -q -O 'locationdata.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [8]:
locs=pd.read_csv('locationdata.csv')
locs.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
merged=final_table.merge(locs,left_on='Postcode',right_on='Postal Code', how='left')
merged.drop('Postal Code',axis=1,inplace=True)
merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [0]:
from geopy.geocoders import Nominatim 



In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [0]:
import folium

In [13]:
toronto=merged[merged['Borough'].str.contains('Toronto')]
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


**Part 3.**

In this part the locations of toronto are parsed form the ontareo data and grouped togther in a map, then venus of different neighborhoods are shown for each location and clustered to divide the neighborhoods into different segments depending on the type of venus in the area.

**Mapping Toronto with its Neighbourhoods and different parts of the city**

Choosing only Neighbourhoods that are located only in Toronto, we plot every Neighbourhood as a point with its information as a interavtive popup in the following map. 

In [14]:
toronto_map= folium.Map(location=[latitude,longitude],zoom_start=12)

for lat, lng, neig,borog in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood'],toronto['Borough']):
    label = '{}, {}'.format(neig, borog)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

**Foursquare**
After finding the locations and ploting them on the map, we use the Foursquare API to find relative information about each location ( venus availabe, ratings ..etc)

In [15]:
CLIENT_ID = '34EHRE4OXKRURVUISQ5WYOWED0CW5LEKS5MAQCTIYEMX2GVS' # your Foursquare ID
CLIENT_SECRET = '2NVR4UDMB0QQIBUWLPLRLWW5WG3XBQJDSCKG51LBDVPUR2SR' # your Foursquare Secret
VERSION = '20190111' # Foursquare API version
LIMIT=100
redius=500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 34EHRE4OXKRURVUISQ5WYOWED0CW5LEKS5MAQCTIYEMX2GVS
CLIENT_SECRET:2NVR4UDMB0QQIBUWLPLRLWW5WG3XBQJDSCKG51LBDVPUR2SR


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )



The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [18]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",42,42,42,42,42,42
Central Bay Street,87,87,87,87,87,87
"Chinatown,Grange Park,Kensington Market",90,90,90,90,90,90
Christie,17,17,17,17,17,17


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [0]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
# fixed_columns = ['Neighborhood'] + list(toronto_onehot.columns[:-1])
# toronto_onehot = toronto_onehot[fixed_columns]




In [22]:
col=list(toronto_onehot.columns)
col.remove('Neighborhood')
toronto_onehot=toronto_onehot[['Neighborhood']+col]
toronto_onehot


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,...,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1706,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1707,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1708,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,...,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,0.010000,0.0,0.03,0.0,0.0000,0.000000,0.0,0.000000,0.020000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.0000,0.0,0.01,0.03,0.010000,...,0.0,0.000000,0.010000,0.0,0.010000,0.0000,0.0,0.01,0.0,0.0,0.0,0.01,0.000000,0.0,0.000000,0.0,0.040000,0.0,0.0,0.03,0.0,0.0,0.000000,0.0,0.0,0.000000,0.030000,0.01,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.01,0.0000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.017857,0.0,0.00,0.0,0.0000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.000000,0.00,0.000000,0.00,0.017857,0.0000,0.0,0.00,0.00,0.000000,...,0.0,0.000000,0.035714,0.0,0.017857,0.0000,0.0,0.00,0.0,0.0,0.0,0.00,0.017857,0.0,0.000000,0.0,0.035714,0.0,0.0,0.00,0.0,0.0,0.017857,0.0,0.0,0.017857,0.017857,0.00,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.00,0.0000
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.0000,0.000000,0.0,0.000000,0.045455,0.0,0.045455,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.090909,0.0000,0.0,0.00,0.00,0.045455,...,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.045455,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.0625,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0625,0.0,0.00,0.00,0.062500,...,0.0,0.000000,0.000000,0.0,0.000000,0.0625,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0625
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.058824,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.058824,0.00,0.058824,0.00,0.000000,0.0000,0.0,0.00,0.00,0.000000,...,0.0,0.058824,0.000000,0.0,0.000000,0.0000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0000


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Clothing Store,Burger Joint,Sushi Restaurant,Asian Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Café,Bistro
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Bar,Bakery,Restaurant,Intersection,Climbing Gym,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Skate Park,Yoga Studio
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden


In [26]:
from sklearn.cluster import KMeans
kclusters=5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [30]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide,King,Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Clothing Store,Burger Joint,Sushi Restaurant,Asian Restaurant,Hotel
1,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Café,Bistro
2,1,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Bar,Bakery,Restaurant,Intersection,Climbing Gym,Italian Restaurant
3,1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Skate Park,Yoga Studio
4,0,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden


In [31]:
# add clustering labels


toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Asian Restaurant,Pub,Health Food Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Pub,Pizza Place,Liquor Store,Juice Bar
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Fish & Chips Shop,Sushi Restaurant,Sandwich Place,Burger Joint,Burrito Place,Italian Restaurant,Intersection,Liquor Store,Ice Cream Shop,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,Brewery,American Restaurant,Gay Bar,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Construction & Landscaping,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,1,Trail,Asian Restaurant,Pub,Health Food Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
41,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Pub,Pizza Place,Liquor Store,Juice Bar
42,East Toronto,1,Fish & Chips Shop,Sushi Restaurant,Sandwich Place,Burger Joint,Burrito Place,Italian Restaurant,Intersection,Liquor Store,Ice Cream Shop,Fast Food Restaurant
43,East Toronto,1,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,Brewery,American Restaurant,Gay Bar,Seafood Restaurant,Sandwich Place
45,Central Toronto,1,Park,Convenience Store,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Gym,Breakfast Spot,Dumpling Restaurant,Donut Shop
46,Central Toronto,1,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Health & Beauty Service,Diner,Dessert Shop,Mexican Restaurant,Chinese Restaurant,Café
47,Central Toronto,1,Dessert Shop,Sandwich Place,Coffee Shop,Pizza Place,Italian Restaurant,Café,Sushi Restaurant,Gym,Greek Restaurant,Brewery
49,Central Toronto,1,Coffee Shop,Pub,Pizza Place,Liquor Store,Sushi Restaurant,Bagel Shop,Sports Bar,Restaurant,Fried Chicken Joint,Supermarket
51,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Café,Pub,Bakery,Plaza,Pharmacy,Caribbean Restaurant
52,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Bubble Tea Shop,Burger Joint,Café,Men's Store,Mediterranean Restaurant


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,2,Playground,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Park,Construction & Landscaping,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
50,Downtown Toronto,3,Park,Playground,Trail,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
64,Central Toronto,3,Jewelry Store,Trail,Sushi Restaurant,Park,General Entertainment,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4,Garden,Yoga Studio,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
